# Aprendizem Profunda - DNN
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [87]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization, LeakyReLU, Input
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline
)


### Input dos dados

In [88]:
df = pd.read_csv("data/fixed_dataset_final.csv")
df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")


### Função pra gravar em csv

In [89]:
def to_csv(results_df, name):
    os.makedirs("score", exist_ok=True)
    path = f"score/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [90]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [91]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 4053, Unique texts: 4051
Overlap between train and test after deduplication: 0


### Tokenizer

In [92]:
# Tokenize the data
tokenizer = Tokenizer(num_words=25000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=200)
X_val = pad_sequences(X_val_seq, maxlen=200)
X_test = pad_sequences(X_test_seq, maxlen=200)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [93]:
data_set_professor = pd.read_csv("data/dataset1_inputs.csv", sep='\t', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=200)  

### Early Stopping

In [94]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=20,            
    restore_best_weights=True,           
)

### Reduce_lr

In [95]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3,
    min_lr=0.00005
)

## DNN

In [96]:
model_dnn = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(64, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(32, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(1, activation="sigmoid"),
])

# Compile
model_dnn.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

# Train
history = model_dnn.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    shuffle=True
)

Epoch 1/100


c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5409 - loss: 1.0747 - val_accuracy: 0.5833 - val_loss: 0.9466 - learning_rate: 1.0000e-04
Epoch 2/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5824 - loss: 0.9949 - val_accuracy: 0.6574 - val_loss: 0.9153 - learning_rate: 1.0000e-04
Epoch 3/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6250 - loss: 0.9577 - val_accuracy: 0.6975 - val_loss: 0.9002 - learning_rate: 1.0000e-04
Epoch 4/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6254 - loss: 0.9608 - val_accuracy: 0.6806 - val_loss: 0.8904 - learning_rate: 1.0000e-04
Epoch 5/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6548 - loss: 0.9180 - val_accuracy: 0.6898 - val_loss: 0.8831 - learning_rate: 1.0000e-04
Epoch 6/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6671 - loss: 0.9037 - val_accuracy: 0.6944 - val_loss: 0.8757 - learning_rate: 1.0000e-04
Epoch 7/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6623 - loss: 0.9

## DNN

# Correr Modelos

**Test Set**

In [97]:
test_ids = df.loc[test_indices, "ID"]  # Get IDs for test split only
predict_and_save(model_dnn, X_test, test_ids, "dnn_dataset1")
model_dnn.save("models/dnn.keras")  

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


**Avaliação**

In [98]:
predict_and_save(model_dnn, X_professor, data_set_professor["ID"], "dnn_dataset1")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [99]:
# 1. Carregar os arquivos
preds_df = pd.read_csv("score/dnn_dataset1.csv", sep='\t', on_bad_lines='skip')
labels_df = pd.read_csv("data/dataset1_outputs.csv", sep='\t', on_bad_lines='skip')

# 2. Garantir que as colunas corretas estão sendo usadas
# Assumindo que a coluna de previsões se chama 'Pred' e a de verdade se chama 'Label'
y_pred = preds_df['Label']
y_true = labels_df['Label']

# 3. Comparar previsões com os rótulos reais
print("Acurácia:", accuracy_score(y_true, y_pred))
print("Relatório de Classificação:")
print(classification_report(y_true, y_pred))

Acurácia: 0.5666666666666667
Relatório de Classificação:
              precision    recall  f1-score   support

          AI       0.75      0.20      0.32        15
       Human       0.54      0.93      0.68        15

    accuracy                           0.57        30
   macro avg       0.64      0.57      0.50        30
weighted avg       0.64      0.57      0.50        30



## Classification Reports

In [100]:
print(classification_report(y_test, model_dnn.predict(X_test) > 0.5))

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

         0.0       0.82      0.56      0.66       420
         1.0       0.65      0.86      0.74       391

    accuracy                           0.71       811
   macro avg       0.73      0.71      0.70       811
weighted avg       0.73      0.71      0.70       811

